In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_absolute_error, mean_squared_error
from IPython.display import display

# Memuat file Excel
file_path = 'data_full.xlsx'  # Ganti dengan path file Anda
df_full_data = pd.read_excel(file_path)

# Menampilkan data yang telah dimuat
display(df_full_data.head())


In [ ]:
# Mengubah data dari format wide ke long
df_long = pd.melt(df_full_data, id_vars=['Wilayah', 'Kategori'], var_name='Minggu', value_name='Pemakaian')

# Mengubah kolom 'Minggu' menjadi numerik
df_long['Minggu'] = df_long['Minggu'].astype(int)

# Menampilkan data dalam format long
display(df_long.head())


In [ ]:
# Mendapatkan daftar wilayah
wilayah_list = df_long['Wilayah'].unique()

# Mendapatkan daftar kategori
kategori_list = df_long['Kategori'].unique()

# Menampilkan daftar wilayah dan kategori
print("Daftar Wilayah:", wilayah_list)
print("Daftar Kategori:", kategori_list)


In [18]:
# Metrik kesalahan per wilayah dan kategori
error_metrics = {}

# Menyimpan hasil per wilayah dan kategori
df_results = {}

# Model yang akan digunakan
models = ['MA (3-Mingguan)', 'AR(1)', 'AR+MA', 'VARMA']


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Fungsi untuk menghitung Moving Average
def moving_average(series, period=3):
    return pd.Series(series).rolling(window=period).mean().values

# Menyimpan hasil dan metrik kesalahan
df_results = {}
error_metrics = {}

# Inisialisasi DataFrame untuk menyimpan semua prediksi
prediksi_final = pd.DataFrame()

for wilayah in wilayah_list:
    # Memfilter data untuk wilayah tertentu
    df_wilayah = df_long[df_long['Wilayah'] == wilayah].copy()
    
    # Mengecek apakah data tersedia
    if df_wilayah.empty:
        continue  # Jika tidak ada data, lewati iterasi ini
    
    # Mengurutkan data berdasarkan 'Minggu' dan 'Kategori'
    df_wilayah.sort_values(['Minggu', 'Kategori'], inplace=True)
    df_wilayah.reset_index(drop=True, inplace=True)
    
    # Pivot data sehingga setiap kategori menjadi kolom
    df_pivot = df_wilayah.pivot(index='Minggu', columns='Kategori', values='Pemakaian').reset_index()
    
    # Mengisi nilai NaN jika ada
    df_pivot.fillna(0, inplace=True)
    
    # Menentukan Kategori yang Tersedia dalam df_pivot
    available_categories = [kategori for kategori in kategori_list if kategori in df_pivot.columns]
    
    # Jika tidak ada kategori yang tersedia, lewati wilayah ini
    if not available_categories:
        print(f"Tidak ada kategori yang tersedia untuk wilayah {wilayah}.")
        continue
    
    # Menyimpan 'Pemakaian Asli' per kategori yang tersedia
    pemakaian_asli = df_pivot[available_categories].values  # Bentuknya (jumlah_minggu, jumlah_kategori)
    
    # Inisialisasi DataFrame hasil untuk wilayah ini
    df_region_results = df_pivot[['Minggu']].copy()
    
    # Untuk setiap kategori yang tersedia, lakukan analisis time series
    for idx, kategori in enumerate(available_categories):
        # Ambil data pemakaian asli untuk kategori ini
        series = pemakaian_asli[:, idx]
        
        # Cek apakah data tidak semuanya nol
        if np.all(series == 0):
            continue  # Jika semua data nol, lewati kategori ini
        
        # Menambahkan pemakaian asli ke df_region_results
        df_region_results[f'{kategori}'] = series
        
        # Menghitung Moving Average
        ma_values = moving_average(series, period=3)
        df_region_results[f'{kategori} - MA (3-Mingguan)'] = ma_values
        
        # Model AR
        phi_ar = 0.8  # Nilai phi untuk model AR(1)
        ar_values = np.append([np.nan], series[:-1] * phi_ar)
        df_region_results[f'{kategori} - AR(1)'] = ar_values
        
        # Model AR+MA
        theta_ma = 0.2  # Nilai theta untuk model MA(1)
        residuals = series - ma_values
        residuals[np.isnan(residuals)] = 0  # Mengganti NaN dengan 0
        ar_ma_values = ar_values + theta_ma * np.append([0], residuals[:-1])
        df_region_results[f'{kategori} - AR+MA'] = ar_ma_values
        
        # Model VARMA
        phi_varma = 1.0298
        theta_varma = 4.445
        beta1 = 0.5
        beta2 = 0.2
        
        def calculate_varma_values(series, residuals, phi, theta, beta1, beta2):
            varma_values = np.full(len(series), np.nan)
            for t in range(len(series)):
                if t < 3:
                    continue  # Biarkan sebagai NaN
                else:
                    varma_value = (phi * series[t-1] + theta * residuals[t-1] +
                                   beta1 * series[t-2] +
                                   beta2 * series[t-3])
                    varma_values[t] = varma_value
            return varma_values
        
        varma_values = calculate_varma_values(series, residuals, phi_varma, theta_varma, beta1, beta2)
        df_region_results[f'{kategori} - VARMA'] = varma_values

        # Menghitung MAPE, RMSE, dan MAE per kategori
        error_metrics_model = {}
        for model in ['MA (3-Mingguan)', 'AR(1)', 'AR+MA', 'VARMA']:
            model_col = f'{kategori} - {model}'
            actual = df_region_results[kategori][~np.isnan(df_region_results[model_col])]
            predicted = df_region_results[model_col][~np.isnan(df_region_results[model_col])]
            if len(actual) > 0:  # Pastikan ada data untuk menghitung metrik
                mae = mean_absolute_error(actual, predicted)
                rmse = mean_squared_error(actual, predicted, squared=False)
                mape = np.mean(np.abs((actual - predicted) / actual)) * 100
                error_metrics_model[model] = {'MAE': round(mae, 2), 'RMSE': round(rmse, 2), 'MAPE': round(mape, 2)}
        
        # Menyimpan metrik kesalahan per kategori
        error_metrics[(wilayah, kategori)] = error_metrics_model
    
    # Menyimpan hasil per wilayah
    df_results[wilayah] = df_region_results.copy()

    # **Prediksi untuk Minggu 53 hingga 62**
    for week in range(53, 63):  # Dari minggu 53 hingga 62
        new_data = {'Minggu': week}
        
        for kategori in available_categories:
            series = df_region_results[kategori].values
            last_value = series[-1]
            
            # Contoh prediksi yang lebih realistis
            ar_ma_pred = max(0, last_value * (1 + np.random.uniform(-0.1, 0.1)))  # Menggunakan noise kecil
            
            new_data[kategori] = ar_ma_pred
        
        # Tambahkan hasil prediksi ke DataFrame
        df_region_results = pd.concat([df_region_results, pd.DataFrame([new_data])], ignore_index=True)

    # Menampilkan hasil prediksi
    prediksi_df = df_region_results[df_region_results['Minggu'].isin(range(53, 63))].copy()
    prediksi_df['Region'] = wilayah
    prediksi_final = pd.concat([prediksi_final, prediksi_df], ignore_index=True)

# Menampilkan tabel akhir hasil prediksi
print("Hasil Prediksi untuk Setiap Wilayah:")
display(prediksi_final)

# Menampilkan grafik untuk setiap wilayah dalam bentuk grid
num_wilayah = len(wilayah_list)
fig, axes = plt.subplots(nrows=num_wilayah, ncols=2, figsize=(15, 5 * num_wilayah))

for i, wilayah in enumerate(wilayah_list):
    df_region = df_results[wilayah].copy()

    # Grafik untuk minggu 1 hingga 52 (data asli)
    axes[i, 0].set_title(f'Data Asli Wilayah {wilayah} (Minggu 1-52)')
    for kategori in available_categories:
        if kategori in df_region.columns:
            axes[i, 0].plot(df_region['Minggu'], df_region[kategori], label=f'Pemakaian {kategori}')
    
    axes[i, 0].set_xlabel('Minggu')
    axes[i, 0].set_ylabel('Pemakaian')
    axes[i, 0].legend()
    axes[i, 0].grid()

    # Grafik untuk minggu 53 hingga 62 (prediksi)
    prediksi_region = prediksi_final[prediksi_final['Region'] == wilayah]
    axes[i, 1].set_title(f'Prediksi Wilayah {wilayah} (Minggu 53-62)')
    for kategori in available_categories:
        if kategori in prediksi_region.columns:
            axes[i, 1].plot(prediksi_region['Minggu'], prediksi_region[kategori], label=f'Prediksi {kategori}', marker='o')
    
    axes[i, 1].set_xlabel('Minggu')
    axes[i, 1].set_ylabel('Pemakaian')
    axes[i, 1].legend()
    axes[i, 1].grid()

plt.tight_layout()
plt.show()


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Membuat subplot untuk setiap wilayah dengan format 2x2
cols = 2
rows = (len(wilayah_list) + cols - 1) // cols  # Menghitung jumlah baris yang diperlukan

fig = make_subplots(
    rows=rows, 
    cols=cols,
    subplot_titles=wilayah_list
)

# Warna yang akan digunakan untuk setiap kategori
colors = {
    'Industri': 'blue',
    'Lain-lain': 'orange',
    'Rumah Tangga': 'green',
    'Sosial': 'red'
}

# Menambahkan data prediksi ke subplot
for idx, wilayah in enumerate(wilayah_list):
    prediksi_region = prediksi_final[prediksi_final['Region'] == wilayah]
    
    # Menentukan posisi subplot
    row = idx // cols + 1
    col = idx % cols + 1
    
    for kategori in available_categories:
        if kategori in prediksi_region.columns:
            # Menentukan apakah jejak ini adalah yang pertama untuk kategori ini
            show_legend = True if (idx == 0) else False  # Hanya menampilkan legend untuk kategori pertama
            
            # Menambahkan jejak hanya jika kategori memiliki nilai
            fig.add_trace(go.Scatter(
                x=prediksi_region['Minggu'],
                y=prediksi_region[kategori],
                mode='lines+markers',
                name=kategori,  # Menyederhanakan nama kategori
                line=dict(color=colors[kategori]),  # Menggunakan warna sesuai kategori
                marker=dict(size=8),
                showlegend=show_legend  # Menentukan apakah legend ditampilkan
            ), row=row, col=col)

# Menyusun layout grafik
fig.update_layout(
    title='Hasil Prediksi Pemakaian Listrik (Minggu 53-62) per Wilayah',
    xaxis_title='Minggu',
    yaxis_title='Pemakaian',
    legend_title='Kategori',
    template='plotly_white',
    height=400 * rows  # Menyesuaikan tinggi berdasarkan jumlah wilayah
)

# Menampilkan garis grid
fig.update_xaxes(showgrid=True, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridcolor='LightGrey')

# Menampilkan grafik
fig.show()


In [ ]:
import matplotlib.pyplot as plt

# Membuat subplot untuk setiap wilayah, dengan kategori sebagai subplots
for wilayah in wilayah_list:
    # Dapatkan daftar kategori yang tersedia untuk wilayah ini
    kategori_wilayah = [kategori for kategori in kategori_list if (wilayah, kategori) in df_results]
    num_kategori = len(kategori_wilayah)
    
    if num_kategori == 0:
        print(f"Tidak ada kategori untuk wilayah {wilayah}.")  # Jika tidak ada kategori, cetak pesan
        continue  # Jika tidak ada kategori untuk wilayah ini, lewati
    
    # Tentukan jumlah baris dan kolom untuk subplots
    cols = 2  # Misalnya, kita gunakan 2 kolom
    rows = (num_kategori + cols - 1) // cols  # Menghitung jumlah baris yang diperlukan
    
    # Membuat figure dan subplots
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()  # Meratakan array axes untuk mempermudah akses
    
    # Loop melalui kategori untuk wilayah ini
    for idx, kategori in enumerate(kategori_wilayah):
        key = (wilayah, kategori)
        df_region = df_results[key]
        
        # Menambahkan Pemakaian Asli
        axes[idx].plot(df_region['Minggu'], df_region['Pemakaian'], marker='o', color='blue', label='Pemakaian Asli')
        
        # Menambahkan model-model
        colors = {'MA (3-Mingguan)': 'orange', 'AR(1)': 'green', 'AR+MA': 'red', 'VARMA': 'purple'}
        for model in models:
            if model in df_region.columns:  # Memeriksa apakah model ada dalam DataFrame
                axes[idx].plot(df_region['Minggu'], df_region[model], marker='o', label=model, color=colors[model])
        
        # Menambahkan judul dan label sumbu pada subplot
        axes[idx].set_title(f'Perbandingan {kategori} di {wilayah}')
        axes[idx].set_xlabel('Minggu')
        axes[idx].set_ylabel('Pemakaian Listrik')
        axes[idx].legend()
    
    # Menghapus subplot kosong jika ada
    for j in range(num_kategori, len(axes)):
        fig.delaxes(axes[j])
    
    # Menyesuaikan layout keseluruhan
    plt.tight_layout()
    plt.suptitle(f'Perbandingan Model untuk Wilayah {wilayah}', fontsize=16)
    plt.subplots_adjust(top=0.9)  # Menyesuaikan jarak judul utama
    plt.show()


In [ ]:
# Menggabungkan semua hasil ke dalam satu DataFrame
combined_results = pd.DataFrame()

for wilayah in df_results:
    df_region = df_results[wilayah].copy()
    df_region['Wilayah'] = wilayah
    df_region.reset_index(drop=True, inplace=True)
    
    # Mendapatkan daftar kolom
    columns = df_region.columns.tolist()
    columns.remove('Minggu')
    columns.remove('Wilayah')
    
    # Mengekstrak kategori dari nama kolom
    categories = set()
    for col in columns:
        base = col.split(' - ')[0]
        categories.add(base)
    
    # Loop melalui setiap kategori
    for kategori in categories:
        # Memilih kolom untuk kategori ini
        cols_to_select = ['Minggu', 'Wilayah']
        kategori_cols = [col for col in df_region.columns if col.startswith(kategori)]
        cols_to_select.extend(kategori_cols)
        
        df_kategori = df_region[cols_to_select].copy()
        df_kategori['Kategori'] = kategori
        
        # Mengganti nama kolom
        rename_dict = {f'{kategori}': 'Pemakaian Asli'}
        for model in models:
            old_col = f'{kategori} - {model}'
            if old_col in df_kategori.columns:
                rename_dict[old_col] = model
        df_kategori.rename(columns=rename_dict, inplace=True)
        
        # Mengubah DataFrame menjadi format long
        df_melted = df_kategori.melt(id_vars=['Minggu', 'Wilayah', 'Kategori'], value_vars=['Pemakaian Asli'] + models,
                                     var_name='Model', value_name='Nilai')
        
        # Menambahkan hasil ke combined_results
        combined_results = pd.concat([combined_results, df_melted], ignore_index=True)

# Menampilkan DataFrame gabungan
display(combined_results)


In [ ]:
# Menggabungkan metrik kesalahan ke dalam DataFrame
error_metrics_list = []

for key, metrics in error_metrics.items():
    wilayah, kategori = key
    for model, values in metrics.items():
        temp_dict = {'Wilayah': wilayah, 'Kategori': kategori, 'Model': model}
        temp_dict.update(values)
        error_metrics_list.append(temp_dict)

error_metrics_df = pd.DataFrame(error_metrics_list)

# Menampilkan metrik kesalahan
display(error_metrics_df)
